## Momentum

Momentum-based Trading is based on the assumption that Stocks which have performed will in the past, will perform better in the coming future.

To define 'past', we take a variable **N**, and say that :

<centre> Momentum(For A particular stock) = Close Price(Today) - Close Price(N-day ago) </centre>

This gives us our first hyper-parameter (parameters of the model which could be changed in order to improve the model) : **N**

We would also be reshuffling our [Portfolio](https://www.investopedia.com/terms/p/portfolio.asp) at certain intervals of time, which gives us our second hyper-parameter: **T** (The time after which we'll be reshuffling our Portfolio)

Its never suggested to keep all your money invested, you must have some risk-free assets as well, so that even if you lose some of your cash in trading, you could still place better bets and regain that lost cash, Thus, We get our third Hyper-parameter: **R**, The Ratio of Total Balance, which we will using for investing.

You will not be investing in all the 30 Tickers now, Will you? You will choose the top few stocks, which show the highest promise in terms of Momentum, which brings us to another hyper-parameter: **M**, The Number of Top few stocks (based on Momentum), which you'll keep in your Portfolio.

Finally, There's some brokerage fee which you need to pay in order to place orders on the stock market, typically its less than 0.05% of the total amount : **F**


In [4]:
#Importing Required Libraries
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

#Declaring the Hyperparameters

N = 50
T = 7
R = 0.8
M = 5
F = 0.0005   # 0.5% Brokerage fee

The Second step would be to define a function which reads the Prices of various Stocks into memory.

In the file DATA.csv , which we had uploaded in our repository, we have prices of 30 firms enlisted in S & P 500 Index (Apple, IBM, Cisco, Walmart and the like!) from 2nd January 2009 to 17th August 2020.

For our purposes, We'll only be requiring certain columns. On an honest note, Just getting the Columns on Ticker, Date and Adjusted Closing Price would do the job, but if you want, you may take Opening Price as well.

Read up about the [pandas.read_csv](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) function from here, and figure out how you'll use it to do the job (You don't need all the columns!)

In [11]:
def GetData(NameOfFile):
    # Read the CSV file
    df = pd.read_csv('data.csv')

    # Select the required columns (Ticker, Date, Adjusted Close)
    required_columns = ['tic', 'datadate', 'adjcp']
    df = df[required_columns]

    return df

To aid Data-Manipulation, it would be beneficial, if we split the DataFrame into many small parts each corresponding to the data corresponding to the 30 Tickers on a particular date. These small parts could then be stored in a list.

We would also be needing to remember which date is at what index, so that we can use that later.

In [12]:
def PartitionData(Data):
    DateToIndex = {}
    DataPartitioned = []

    # Group the data by date
    grouped_data = Data.groupby('datadate')

    # Iterate over unique dates
    for date, group in grouped_data:
        # Store the index of the date
        DateToIndex[date] = len(DataPartitioned)

        # Append the data for the date to the partitioned list
        DataPartitioned.append(group)

    return DataPartitioned, DateToIndex

Now, We need a function to calculate the Momentum value for all of our 30 Tickers.
To do this, We need to have a few things in mind:


1.   We need to start at Nth day in our list, as only then we'll be able to calculate the Momentum (This will be taken care of by later parts of the Program, when we actually run the Model)

2.   The Stock Market isn't open on all days, so we often won't be able to go N days behind, and will have to take the closest value instead(We can't just go N entries behind in the List we created and expect that to work, Why?) In order to work with dates, you should get to know more about the datetime library of Python from [here](https://thispointer.com/python-how-to-convert-datetime-object-to-string-using-datetime-strftime/) (Especially the datetime.strftime() function) and about the [datetime.timedelta()](https://www.studytonight.com/python-howtos/how-to-add-days-to-date-in-python) function.

Also, as you may have figured it out yourself, while DataFrames are great for Data Handling and small statistical calculations, They aren't so for big calculations as the Numpy Library has both a richer variety of functions for such manipulations and is also more efficient!

After we calculate the Momentum for all our Tickers, it would be a great thing to do, if we could divide their prices by their mean(in the N day interval, as we need to see which stock outperforms others and in order to do this, it won't be fair if we take the absolute growth in numbers!(Why?)



In [81]:
import numpy as np

def GetMomentumBasedPriority(PartitionedDataFrameList, DateToIndex, today, N):
    # Convert today to string representation of the date
    today_str = today.strftime("%Y%m%d")

    # Initialize an empty list to store momentum-based priority
    momentum_priority = []

    # Iterate over the PartitionedDataFrameList
    for ticker in PartitionedDataFrameList:
        # Get the index of today's date
        index = DateToIndex[today_str]

        # Get the data for the ticker on the N days ago date
        ticker_data = ticker.iloc[index - N]

        # Calculate momentum: Close Price(Today) - Close Price(N days ago)
        momentum = ticker_data['Adj Close'] - ticker_data['Adj Close'].iloc[-N]

        # Calculate mean in the N-day period considered
        mean = ticker_data['Adj Close'].mean()

        # Divide momentum by mean and append to the priority list
        momentum_priority.append(momentum / mean)

    return np.array(momentum_priority)

Even after you have got your Momentum-based priorities, and have decided which stocks to buy and what will be the weight of each, you still need to figure out how much of each will you buy. To do this, first you'll sell all your pre-owned stocks which will increase your cash in hand, then you'll know the stocks to buy and their relative weights (given by their Momentum/mean) and you need a function which tells you how many stocks to buy for each ticker!

In [82]:
def GetBalanced(prices, weights, balance):
    # prices: Numpy array containing Prices of all the 30 Stocks
    # weights: Multi-hot Numpy Array: The Elements corresponding to stocks which are to be bought
    #          (Top M Stocks with positive Momentum Indicator) are set to their priority,
    #          All other elements are set to zero.
    # balance: Total available balance for investment

    # Calculate the total weight of selected stocks
    total_weight = np.sum(weights)

    # Calculate the allocation ratio for each stock
    allocation_ratio = weights / total_weight

    # Calculate the allocated amount for each stock
    allocated_amount = balance * allocation_ratio

    # Calculate the number of shares to buy for each stock
    shares_to_buy = allocated_amount / prices

    return shares_to_buy

Now, We need something to simulate our [Portfolio](https://www.investopedia.com/terms/p/portfolio.asp). In order to do that we need a class, which has certain  basic features and functionalities.

Features :


1.   Your Initial Balance
2.   Your Current Balance
3.   A list(/any other container) storing the number of stocks of each ticker currently in possession. (Numpy Array prefered)
4.   Most recent prices of all the stocks (As a Numpy array)

Functionalities:



1.   Calculating current Net Worth (Balance+Total Evaluation of all Stocks owned!)
2.   Buying a Particular Stock (Keep the Transaction fee in mind!)
3.   Selling a particular Stock whole (Keep the Transaction Fee in mind!)
4.   Rebalance Portfolio  (Takes Numpy array as input)
5.   Function to change the value of most recent prices stored (Takes Numpy array as input)





In [83]:
class PortFolio:
    def __init__(self, initial_balance, initial_stock_holdings, initial_prices):
        self.initial_balance = initial_balance
        self.current_balance = initial_balance
        self.stock_holdings = np.array(initial_stock_holdings)
        self.prices = np.array(initial_prices)

    def SellStock(self, index):
        # index: The index of the Stock to sell (0-29)

        # Check if the stock is already owned
        if self.stock_holdings[index] > 0:
            # Calculate the transaction fee
            transaction_fee = F * self.prices[index] * self.stock_holdings[index]

            # Calculate the proceeds from selling the stock
            proceeds = self.prices[index] * self.stock_holdings[index] - transaction_fee

            # Update the current balance and stock holdings
            self.current_balance += proceeds
            self.stock_holdings[index] = 0

    def BuyStock(self, index, number):
        # index: The index of the Stock to buy (0-29)
        # number: Number of shares to buy (float)

        # Calculate the transaction cost
        transaction_cost = self.prices[index] * number + F * self.prices[index] * number

        # Check if the current balance is sufficient to buy the stock
        if self.current_balance >= transaction_cost:
            # Update the current balance and stock holdings
            self.current_balance -= transaction_cost
            self.stock_holdings[index] += number

    def CalculateNetWorth(self):
    # Reshape stock_holdings array to match prices shape
        reshaped_stock_holdings = self.stock_holdings.reshape((30,))

    # Return Net Worth (All Shares' costs + Balance)
        stock_costs = np.sum(reshaped_stock_holdings * self.prices)
        net_worth = self.current_balance + stock_costs
        return net_worth

    def ChangePricesTo(self, newPriceVector):
        # newPriceVector: Pandas DataFrame containing the prices of all the stocks for the current day
        self.prices = pd.to_numeric(newPriceVector.iloc[:, 0], errors='coerce').values  # Extract the first column as prices and convert to numeric

    def RebalancePortFolio(self, newWeights):
        # newWeights: Numpy array containing Momentum/Mean for all stocks in the N-day period

        # Sell all pre-owned stocks
        for i in range(len(self.stock_holdings)):
            self.SellStock(i)

        # Determine the stocks to buy based on the top M stocks with positive momentum
        positive_momentum_indices = np.argsort(-newWeights)[:M]
        positive_momentum_weights = newWeights[positive_momentum_indices]

        # Rebalance the portfolio using GetBalanced function
        shares_to_buy = GetBalanced(self.prices, positive_momentum_weights, self.current_balance)

        # Buy the stocks
        for i in range(len(positive_momentum_indices)):
            self.BuyStock(positive_momentum_indices[i], shares_to_buy[i])

With that the difficult part is over!

Now, all you need to work on is a main loop, which calls all these functions

In [84]:
import numpy as np
import pandas as pd

myPortfolio = PortFolio(initial_balance=100000, initial_stock_holdings=[0.0] * 30, initial_prices=np.zeros((30,)))

NetWorthAfterEachTrade = []

# First Get the Data
Data = GetData('data.csv')
PartitionedData, DateToIndex = PartitionData(Data)

# Start processing from the (N+1)th Day (among the ones recorded in the Data)
for i in range(N, len(PartitionedData)):
    # Change the Prices to the ith Term
    myPortfolio.ChangePricesTo(PartitionedData[i])

    # Get NetWorth and store in list
    net_worth = myPortfolio.CalculateNetWorth()
    NetWorthAfterEachTrade.append(net_worth)

    # Check if you need to rebalance Portfolio today
    if i % T == 0:
        # Calculate the Momentum-based priority
        today_index = DateToIndex.get(i)  # Check if i exists as a key in DateToIndex dictionary
        if today_index is not None:
            today = PartitionedData[today_index].index[0]  # Get the actual date from the dataframe index
            momentum_priority = GetMomentumBasedPriority(PartitionedData, DateToIndex, today, N)

            # Rebalance the portfolio
            myPortfolio.RebalancePortFolio(momentum_priority)

##Moment of Truth

Time to check, if your Program actually works!

Plot the data you collected in various ways and see if what you did worked!

Feel free to use whichever one of Matplotlib or Seaborn you want to.

You should try changing the hyper-parameters to increase(/decrease) your performance!


In [85]:
import matplotlib.pyplot as plt

def VizualizeData(net_worth_data, prices_data):
    # Plot Net Worth
    plt.figure(figsize=(10, 5))
    plt.plot(net_worth_data)
    plt.xlabel('Time')
    plt.ylabel('Net Worth')
    plt.title('Net Worth Over Time')
    plt.show()

    # Plot Stock Prices
    plt.figure(figsize=(10, 5))
    for i, prices in enumerate(prices_data):
        plt.plot(prices, label=f'Ticker {i+1}')
    plt.xlabel('Time')
    plt.ylabel('Price')
    plt.title('Stock Prices Over Time')
    plt.legend()
    plt.show()

In [86]:
VizualizeData(NetWorthAfterEachTrade, [data.iloc[:, 0].values for data in PartitionedData])

Output hidden; open in https://colab.research.google.com to view.

You may use this cell to write about what results you got!